In [109]:
import pandas as pd
import re

In [110]:
df = pd.read_excel('Unstructured.xlsx')

In [111]:
df

,Emails
0,An internal memo revealed testing issues that ...
1,"The agency reported that CS2 had over 1,000 re..."
2,software failures ended up costing an estimate...
3,"EDS, a Texas-based contractor, also announced ..."
4,"As result of software failures, thousands of t..."
5,Baggage handling system failed miserably in re...
6,"At the end of a busy day, the manager receives..."
7,"For missed EOVS notice, Technology Manager rec..."
8,"Due to Insufficient backup power, there is a 7..."
9,Disk drive failure caused Operating System ero...


In [112]:
# Define patterns for technology events
software_failures = r"(software failure|buggy software|EOL)"
hardware_failures = r"(hardware failure|disk drive failure|fire)"
security_issues = r"(security|vendor support|user accounts|passwords|fire)"

In [113]:
# Create empty dictionary to store event counts
event_counts = {"Software Failure": 0, "Hardware Failure": 0, "Security Issue": 0}

In [114]:
# Function to identify and count events
def count_events(text, patterns, event_counts):
  for event_type, pattern in patterns.items():
    matches = len(re.findall(pattern, text, re.IGNORECASE))
    event_counts[event_type] += matches
  return event_counts

In [115]:
# Apply function to each email in the DataFrame
df["Events"] = df["Emails"].apply(lambda email: count_events(email, 
                                                             {"Software Failure": software_failures, 
                                                              "Hardware Failure": hardware_failures, 
                                                              "Security Issue": security_issues}, 
                                                             event_counts.copy()))


In [116]:
# Summarize event counts
event_summary = pd.DataFrame.from_dict(event_counts, orient='index', columns=['Count'])

In [117]:
# Display results
df["Events"].iloc[1]

{'Software Failure': 2, 'Hardware Failure': 0, 'Security Issue': 0}

In [118]:
df.to_excel('Events2.xlsx')

In [119]:
df['Event'] = df['Events'].apply(lambda x: max(x, key=x.get))

In [120]:
# Drop the original 'Events' column if you no longer need it
df.drop(columns=['Events'], inplace=True)

# Q1. # a No 1 Extract Technology events from the unstructured data

In [121]:
#1.	Extract Technology events from the unstructured data
df

,Emails,Event
0,An internal memo revealed testing issues that ...,Software Failure
1,"The agency reported that CS2 had over 1,000 re...",Software Failure
2,software failures ended up costing an estimate...,Software Failure
3,"EDS, a Texas-based contractor, also announced ...",Software Failure
4,"As result of software failures, thousands of t...",Software Failure
5,Baggage handling system failed miserably in re...,Software Failure
6,"At the end of a busy day, the manager receives...",Software Failure
7,"For missed EOVS notice, Technology Manager rec...",Security Issue
8,"Due to Insufficient backup power, there is a 7...",Hardware Failure
9,Disk drive failure caused Operating System ero...,Hardware Failure


# Q1. # a No 2 Counting events for each category

In [122]:
# Count Events for Each Category
# To count the occurrences of each event category, we can use the value_counts() function on the “Event” column.
event_counts = df['Event'].value_counts()
event_counts


Event
Software Failure    10
Security Issue       6
Hardware Failure     4
Name: count, dtype: int64

# Q1. # a No 3  Calculating the likelihood and impact

In [123]:
# Define function to calculate likelihood based on 'Event'
def calculate_likelihood(event):
    if event == 'Software Failure':
        return 'Medium'
    elif event == 'Security Issue':
        return 'High'
    elif event == 'Hardware Failure':
        return 'Low'
    else:
        return 'Unknown'

In [124]:
# Define function to calculate impact based event
def calculate_impact(event):
    if 'Software Failure' in event:
        return 'High'
    elif 'Security Issue' in event:
        return 'Medium'
    elif 'Hardware Failure' in event:
        return 'Low'
    else:
        return 'Unknown'

In [125]:
# Apply calculate_likelihood and calculate_impact functions to 'Event' column and add new columns to DataFrame
df['Likelihood'] = df['Event'].apply(calculate_likelihood)
df['Impact'] = df['Event'].apply(calculate_impact)

In [126]:
# Display the updated DataFrame
df

,Emails,Event,Likelihood,Impact
0,An internal memo revealed testing issues that ...,Software Failure,Medium,High
1,"The agency reported that CS2 had over 1,000 re...",Software Failure,Medium,High
2,software failures ended up costing an estimate...,Software Failure,Medium,High
3,"EDS, a Texas-based contractor, also announced ...",Software Failure,Medium,High
4,"As result of software failures, thousands of t...",Software Failure,Medium,High
5,Baggage handling system failed miserably in re...,Software Failure,Medium,High
6,"At the end of a busy day, the manager receives...",Software Failure,Medium,High
7,"For missed EOVS notice, Technology Manager rec...",Security Issue,High,Medium
8,"Due to Insufficient backup power, there is a 7...",Hardware Failure,Low,Low
9,Disk drive failure caused Operating System ero...,Hardware Failure,Low,Low


# Q1. # a No.4 Calculate the risk levels and ratings in the attached sheet

In [127]:
# Define risk matrix
risk_matrix = {
    ('Low', 'Low'): {'Risk Level': 'Low'},
    ('Low', 'Medium'): {'Risk Level': 'Low'},
    ('Low', 'High'): {'Risk Level': 'Medium'},
    ('Medium', 'Low'): {'Risk Level': 'Low'},
    ('Medium', 'Medium'): {'Risk Level': 'Medium'},
    ('Medium', 'High'): {'Risk Level': 'High'},
    ('High', 'Low'): {'Risk Level': 'Medium'},
    ('High', 'Medium'): {'Risk Level': 'High'},
    ('High', 'High'): {'Risk Level': 'Very High'}
}

In [128]:
# Function to calculate risk level and rating
def calculate_risk_level(row):
    likelihood = row['Likelihood']
    impact = row['Impact']
    if (likelihood, impact) in risk_matrix:
        return pd.Series(risk_matrix[(likelihood, impact)])
    else:
        return pd.Series({'Risk Level': 'Unknown'})

In [129]:
# Apply function to calculate risk level and rating
df['Risk Level'] = df.apply(calculate_risk_level, axis=1)

In [130]:
# Define a mapping for Likelihood and Impact
likelihood_map = {"Low": 1, "Medium": 2, "High": 3}
impact_map = {"Low": 1, "Medium": 2, "High": 3}

In [131]:
# Calculate the Risk Score
df["Risk Score"] = df.apply(lambda row: likelihood_map[row["Likelihood"]] * impact_map[row["Impact"]], axis=1)

In [132]:
df

,Emails,Event,Likelihood,Impact,Risk Level,Risk Score
0,An internal memo revealed testing issues that ...,Software Failure,Medium,High,High,6
1,"The agency reported that CS2 had over 1,000 re...",Software Failure,Medium,High,High,6
2,software failures ended up costing an estimate...,Software Failure,Medium,High,High,6
3,"EDS, a Texas-based contractor, also announced ...",Software Failure,Medium,High,High,6
4,"As result of software failures, thousands of t...",Software Failure,Medium,High,High,6
5,Baggage handling system failed miserably in re...,Software Failure,Medium,High,High,6
6,"At the end of a busy day, the manager receives...",Software Failure,Medium,High,High,6
7,"For missed EOVS notice, Technology Manager rec...",Security Issue,High,Medium,High,6
8,"Due to Insufficient backup power, there is a 7...",Hardware Failure,Low,Low,Low,1
9,Disk drive failure caused Operating System ero...,Hardware Failure,Low,Low,Low,1


# Q1. # b Performing Linear Regression to get the coefficients and equation of Regression.

In [174]:
df = pd.read_excel('Sample.xlsx')

In [175]:
df.isnull().sum()

Event ID                                                      0
Sub-Event ID                                                  0
Sub-Events' Counts                                            0
Probablity \n(Extract or Default to Medium when not found)    0
Risk Control                                                  0
Likelihood                                                    0
Impact                                                        0
Risk factor                                                   0
dtype: int64

In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 8 columns):
 #   Column                                                     Non-Null Count  Dtype
---  ------                                                     --------------  -----
 0   Event ID                                                   18 non-null     int64
 1   Sub-Event ID                                               18 non-null     int64
 2   Sub-Events' Counts                                         18 non-null     int64
 3   Probablity 
(Extract or Default to Medium when not found)  18 non-null     int64
 4   Risk Control                                               18 non-null     int64
 5   Likelihood                                                 18 non-null     int64
 6   Impact                                                     18 non-null     int64
 7   Risk factor                                                18 non-null     int64
dtypes: int64(8)
memory usage: 1.3 KB


In [177]:
df.describe()

,Event ID,Sub-Event ID,Sub-Events' Counts,Probablity \n(Extract or Default to Medium when not found),Risk Control,Likelihood,Impact,Risk factor
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,2.500000,9.500000,2.500000,4.111111,0.388889,3.055556,2.500000,7.833333
std,1.424574,5.338539,1.150447,1.131833,0.501631,0.937595,1.150447,4.246105
min,1.000000,1.000000,1.000000,2.000000,0.000000,1.000000,1.000000,1.000000
25%,1.000000,5.250000,2.000000,3.000000,0.000000,2.250000,2.000000,4.500000
50%,2.000000,9.500000,2.500000,5.000000,0.000000,3.000000,2.500000,7.000000
75%,3.750000,13.750000,3.000000,5.000000,1.000000,4.000000,3.000000,12.000000
max,5.000000,18.000000,5.000000,5.000000,1.000000,4.000000,5.000000,16.000000


In [179]:
df

,Event ID,Sub-Event ID,Sub-Events' Counts,Probablity \n(Extract or Default to Medium when not found),Risk Control,Likelihood,Impact,Risk factor
0,1,1,2,4,1,2,2,4
1,1,2,3,3,1,3,3,9
2,1,3,2,3,0,3,2,6
3,1,4,3,2,1,4,3,12
4,1,5,4,3,1,4,4,16
5,1,6,5,4,1,2,5,10
6,2,7,1,5,0,3,1,3
7,2,8,3,5,0,2,3,6
8,2,9,4,5,1,3,4,12
9,2,10,2,5,0,3,2,6


In [180]:
X = df[['Likelihood','Impact', 'Risk Control']].values

In [182]:
y = df['Risk factor'].values

In [183]:
# Normalizing the data
from sklearn.preprocessing import MinMaxScaler


In [184]:
scaler = MinMaxScaler()

In [185]:
X

array([[2, 2, 1],
       [3, 3, 1],
       [3, 2, 0],
       [4, 3, 1],
       [4, 4, 1],
       [2, 5, 1],
       [3, 1, 0],
       [2, 3, 0],
       [3, 4, 1],
       [3, 2, 0],
       [4, 2, 0],
       [4, 3, 1],
       [4, 1, 0],
       [2, 1, 0],
       [3, 2, 0],
       [4, 3, 0],
       [4, 3, 0],
       [1, 1, 0]], dtype=int64)

In [186]:
X = scaler.fit_transform(X)

In [187]:
X

array([[0.33333333, 0.25      , 1.        ],
       [0.66666667, 0.5       , 1.        ],
       [0.66666667, 0.25      , 0.        ],
       [1.        , 0.5       , 1.        ],
       [1.        , 0.75      , 1.        ],
       [0.33333333, 1.        , 1.        ],
       [0.66666667, 0.        , 0.        ],
       [0.33333333, 0.5       , 0.        ],
       [0.66666667, 0.75      , 1.        ],
       [0.66666667, 0.25      , 0.        ],
       [1.        , 0.25      , 0.        ],
       [1.        , 0.5       , 1.        ],
       [1.        , 0.        , 0.        ],
       [0.33333333, 0.        , 0.        ],
       [0.66666667, 0.25      , 0.        ],
       [1.        , 0.5       , 0.        ],
       [1.        , 0.5       , 0.        ],
       [0.        , 0.        , 0.        ]])

In [188]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [189]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [190]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [191]:
Y_pred = model.predict(X_test)

In [192]:
# The coefficient 
model.coef_

array([6.48042705, 9.06761566, 1.27758007])

In [193]:
#The intercept
model.intercept_

-0.697508896797153

In [194]:
print("The Equation of regression is y = ", model.coef_[0], " * Likelihood ", model.coef_[1], " * Impact ", model.coef_[2],  " * Risk Level ", " + ", model.intercept_)

The Equation of regression is y =  6.4804270462633395  * Likelihood  9.067615658362993  * Impact  1.2775800711743797  * Risk Level   +  -0.697508896797153


# Q 2 Using the SVM algorithm

In [205]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC

In [195]:
df

,Event ID,Sub-Event ID,Sub-Events' Counts,Probablity \n(Extract or Default to Medium when not found),Risk Control,Likelihood,Impact,Risk factor
0,1,1,2,4,1,2,2,4
1,1,2,3,3,1,3,3,9
2,1,3,2,3,0,3,2,6
3,1,4,3,2,1,4,3,12
4,1,5,4,3,1,4,4,16
5,1,6,5,4,1,2,5,10
6,2,7,1,5,0,3,1,3
7,2,8,3,5,0,2,3,6
8,2,9,4,5,1,3,4,12
9,2,10,2,5,0,3,2,6


In [206]:
# Split data into train and test sets
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

In [208]:
# Write training data to Excel sheet 'TechTrain'
train_set.to_excel('TechTrain.xlsx')

In [209]:
# Write test data to Excel sheet 'TechTest'
test_set.to_excel('TechTest.xlsx')

In [210]:

train = pd.read_excel('TechTrain.xlsx')
test = pd.read_excel('TechTest.xlsx')

In [211]:
# Define features and target variable for the training dataset
X_train = train[["Sub-Event ID", "Sub-Events' Counts", "Risk Control", "Likelihood", "Impact"]]
y_train = train["Risk factor"]

In [212]:
# Define features and target variable for the testing dataset
X_test = test[["Sub-Event ID", "Sub-Events' Counts", "Risk Control", "Likelihood", "Impact"]]
y_test = test["Risk factor"]

In [213]:
# Binarize the target variable (e.g., relevant if Risk factor >= 6) for the training dataset
y_train = (y_train >= 6).astype(int)


In [215]:
# Binarize the target variable (e.g., relevant if Risk factor >= 6) for the testing dataset
y_test = (y_test >= 6).astype(int)


In [216]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [217]:
svm = LinearSVC()
svm.fit(X_train, y_train)

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC()

In [218]:
# Make predictions
y_pred = svm.predict(X_test)

In [ ]:
# Evaluate the model
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))